# COVID-19 Tableau d'Analyse Interactive

>GISinfos Startup: leyepro@hotmail.fr/ Tel: 00221778643290

## C'est quoi le COVID-19?

>Les coronavirus sont des virus à ARN fréquents, de la famille des Coronaviridae, qui sont responsables d'infections digestives et respiratoires chez l'Homme et l'animal. Le virus doit son nom à l'apparence de ses particules virales, portant des excroissances qui évoquent une couronne. Les virions, qui sont constitués d'une capside recouverte d'une enveloppe, mesurent 80 à 150 nm de diamètre.
Les petites sphères contiennent un acide ribonucléique (ARN) monocaténaire (avec une seule chaîne), linéaire (non-segmenté) et positif, comptabilisant 27 à 32 kilobases. Cet ARN se réplique dans le cytoplasme de la cellule infectée.

![Coronavirus particle Image](https://samseinfos.com/wp-content/uploads/2020/04/coronavirus-1.jpg)

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [2]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
confirmed_df;

In [4]:
country_df;

In [5]:
country_df.query('Country_Region == "Cabo Verde"');

In [6]:
afrique = country_df.loc[[117,75,151,115,81,82,49,45]]

In [7]:
afrique.reset_index();

In [8]:
# data cleaning

# renaming the df column names to lowercase
afrique.columns = map(str.lower, afrique.columns);

In [9]:
afrique.sort_values('confirmed', ascending= False).reset_index();

In [10]:
afrique = afrique.rename(columns={'country_region': 'country'})

In [17]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: #FB0202'
    y = 'background-color: #0602FB'
    g = 'background-color: #58FF33'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return afrique.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [12]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['Confirmed'].sum())
deaths_total = int(country_df['Deaths'].sum())
recovered_total = int(country_df['Recovered'].sum())
active_total = int(country_df['Active'].sum())

In [13]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [14]:
sorted_country_df = afrique.sort_values('confirmed', ascending= False)

In [20]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(8), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Statistiques des Pays Africains",
    xaxis_title="Pays",
    yaxis_title="Cas Confirmés",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=8)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=8, description='n', max=24, min=-8), Output()), _dom_classes=('widget-in…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

In [21]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();


In [22]:
interact(plot_cases_of_a_country, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

In [23]:
px.bar(
    sorted_country_df.head(8),
    x = "country",
    y = "confirmed",
    title= "Statistiques des personnes malades", # the axis names
    color_discrete_sequence=["#0602FB"], 
    height=500,
    width=800
)

In [24]:
px.bar(
    sorted_country_df.head(8),
    x = "country",
    y = "deaths",
    title= "Statistiques des personnes décédées", # the axis names
    color_discrete_sequence=["#FB1502"], 
    height=500,
    width=800
)

In [25]:
px.bar(
    sorted_country_df.head(8),
    x = "country",
    y = "recovered",
    title= "Statistiques des personnes guéris", # the axis names
    color_discrete_sequence=["#28F510"], 
    height=500,
    width=800
)

In [26]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['Lat'], confirmed_df.iloc[i]['Long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['Country/Region'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map

## [Notebook covers:](https://github.com/datasciencewithharshit/voila-covid-19-dashboard)

1. C'est quoi le COVID-19?
2. Téléchargement des données sur [John Hopkins CSSE data repository](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)
3. Nettoyage et Préparation des données
4. Visualisation des dnnées en ulisant le package [plotly](https://plotly.com/) scatter plot.
5. Representation des cas de contamination, de décés, et de guérison.
6. Cartographie globale de la situation actuelle avec [Folium](https://python-visualization.github.io/folium/)


## Symptoms:
La COVID-19 n’a pas les mêmes effets en fonction des personnes. La majorité des individus ne ressentiront que des ‎symptômes bénins ou modérés. Les symptômes les plus communs sont:
* de la fièvre,
* une sensation de fatigue,
* une toux sèche. Certaines personnes peuvent également éprouver: 
* des courbatures et des douleurs,
* une congestion nasale,
* un écoulement nasal,
* des maux de gorge,
* des diarrhées.

En moyenne, les symptômes apparaissent 5 ou ‎‎6 jours après qu’une personne a été infectée par le ‎virus, mais cela peut aller jusqu’à 14 jours. ‎
Les personnes qui présentent des symptômes ‎bénins et n’ont pas d’autres problèmes de santé ‎doivent s’isoler. Consultez un médecin si vous avez ‎de la fièvre, que vous toussez et que vous avez des ‎difficultés à respirer. Prévenez le service concerné ‎par téléphone au préalable. ‎

## Pour plus d'information sur le  COVID-19:
* [https://www.who.int/health-topics/coronavirus](https://www.who.int/health-topics/coronavirus)
* [https://www.who.int/emergencies/diseases/novel-coronavirus-2019](https://www.who.int/emergencies/diseases/novel-coronavirus-2019)
* [https://www.nature.com/articles/s41597-020-0448-0](https://www.nature.com/articles/s41597-020-0448-0)

## GISinfos Plateforme
> Cette plateforme est dévelopée par des Géographes Aménagistes Spécialisés en Géomatique et Télédétection Spatiale
Pour plus d'information vous pouvez nous contacter sur:
* leyepro@hotmail.fr
* Tel: 00221778643290

GISinfosTeam